## Regression Model In Keras - Final Project - Part B

In this project I'll build a regression model using the deep learing Keras library, and then experiment with increasing the number of training epochs and changing number of hidden layers and see how changing these parameters impact the performance of the model.

This project is the conclusion of the course 'Introduction to Deep Learning and Neural Networds with Keras' by IBM\Coursera. This is the second part out of 4 where I'll show the diferences reporting the mean and the Standard Deviation of the Mean Squared Error.

In [1]:
# import libraries

import pandas as pd
import numpy as np
import keras

In [2]:
# Downloand the data
# is the same dataset using in the course

concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# checking data points

concrete_data.shape

(1030, 9)

In [4]:
# check for any missing values

concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

### Split data into predictors and target

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
# check the predictors column
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
# check the target column
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

The last step is normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
# saving the num of predictors to n_cols since we'll need theis num when building the network

n_cols = predictors_norm.shape[1] # num of predictors
n_cols

8

In [11]:
# step 1
# split randomly into train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.30, random_state = 42)

print("number of test samples :", X_test.shape[0])
print("number of training samples:",X_train.shape[0])

number of test samples : 309
number of training samples: 721


### Build a neural network
Define a function that defines the regression model so that can conveniently call it to create the model

In [12]:
# importing the rest ov the packages from Keras library that will need for the regression model

from keras.models import Sequential
from keras.layers import Dense

In [13]:
# define regression model

def regression_model():
    # create model
    model = Sequential()
    # one hidden layers and 50 nodes and ReLu activation function
    model.add(Dense(50, activation = 'relu', input_shape = (n_cols,)))
    model.add(Dense(1))
    
    # compile model
    # use 'adam' optimizer and mse as loss function
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model

### Train and Test the Network

Call the function now to create the model

In [14]:
# build the model
model = regression_model()

In [15]:
# step 2
# train the model on train data using 50 epochs

model.fit(X_train, y_train, validation_split = 0.3, epochs = 50, verbose = 2)

Epoch 1/50
16/16 - 1s - loss: 2911.9226 - val_loss: 1540.8599 - 1s/epoch - 72ms/step
Epoch 2/50
16/16 - 0s - loss: 853.7360 - val_loss: 749.1463 - 100ms/epoch - 6ms/step
Epoch 3/50
16/16 - 0s - loss: 494.2130 - val_loss: 473.1965 - 100ms/epoch - 6ms/step
Epoch 4/50
16/16 - 0s - loss: 365.8794 - val_loss: 360.4223 - 82ms/epoch - 5ms/step
Epoch 5/50
16/16 - 0s - loss: 277.2919 - val_loss: 262.6194 - 85ms/epoch - 5ms/step
Epoch 6/50
16/16 - 0s - loss: 215.2111 - val_loss: 213.3764 - 101ms/epoch - 6ms/step
Epoch 7/50
16/16 - 0s - loss: 183.9075 - val_loss: 182.4129 - 82ms/epoch - 5ms/step
Epoch 8/50
16/16 - 0s - loss: 160.4394 - val_loss: 161.9962 - 84ms/epoch - 5ms/step
Epoch 9/50
16/16 - 0s - loss: 148.1872 - val_loss: 150.7646 - 102ms/epoch - 6ms/step
Epoch 10/50
16/16 - 0s - loss: 139.8523 - val_loss: 142.8292 - 79ms/epoch - 5ms/step
Epoch 11/50
16/16 - 0s - loss: 133.0247 - val_loss: 135.3566 - 100ms/epoch - 6ms/step
Epoch 12/50
16/16 - 0s - loss: 129.7090 - val_loss: 130.7797 - 83ms/

In [16]:
# step 3
# evaluate the model 
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 3ms/step


118.12095642089844

In [17]:
# importing mean squared error
from sklearn.metrics import mean_squared_error

mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
mean

118.12095492290894

In [18]:
stdv = np.std(mean_square_error)
stdv

0.0

In [19]:
# step 4
# repeat step 1 - 3, 50 times(creat a list of 50 mse)
import tensorflow

total_mean_squared_errors = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.3, random_state = 42)
    model.fit(X_train, y_train, epochs=50, verbose = 2)
    MSE = model.evaluate(X_test, y_test, verbose = 2)
    print('MSE '+str(i + 1)+': '+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)
    
mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

Epoch 1/50
23/23 - 0s - loss: 115.6814 - 62ms/epoch - 3ms/step
Epoch 2/50
23/23 - 0s - loss: 111.2432 - 56ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 127.1058 - 53ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 110.3437 - 52ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 104.1618 - 57ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 106.0725 - 52ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 105.5515 - 52ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 103.3080 - 56ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 108.8465 - 52ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 104.7749 - 58ms/epoch - 3ms/step
Epoch 11/50
23/23 - 0s - loss: 106.1279 - 53ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 107.3716 - 58ms/epoch - 3ms/step
Epoch 13/50
23/23 - 0s - loss: 102.7925 - 54ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 103.2109 - 57ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 102.8517 - 52ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 103.0389 - 57ms/ep

Epoch 27/50
23/23 - 0s - loss: 69.2069 - 51ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 73.8379 - 51ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 71.3221 - 49ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 71.1880 - 49ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 76.1878 - 48ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 73.3501 - 48ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 70.1409 - 47ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 76.6280 - 49ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 76.2855 - 49ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 71.7497 - 48ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 70.7524 - 47ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 68.4663 - 48ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 69.4654 - 48ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 72.7424 - 49ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 80.9970 - 50ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 77.9451 - 48ms/epoch - 2

23/23 - 0s - loss: 77.4177 - 66ms/epoch - 3ms/step
Epoch 2/50
23/23 - 0s - loss: 68.2979 - 51ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 64.6464 - 49ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 82.0770 - 48ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 81.8256 - 48ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 69.9910 - 49ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 69.5266 - 49ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 62.6120 - 48ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 64.4020 - 48ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 63.9909 - 49ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 68.4390 - 47ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 68.2559 - 105ms/epoch - 5ms/step
Epoch 13/50
23/23 - 0s - loss: 74.2113 - 60ms/epoch - 3ms/step
Epoch 14/50
23/23 - 0s - loss: 77.6800 - 48ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 64.9036 - 47ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 60.8745 - 49ms/epoch - 2ms/step
Epoch 17/50

Epoch 28/50
23/23 - 0s - loss: 55.1154 - 50ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 52.5145 - 47ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 54.6113 - 48ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 64.1848 - 48ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 59.9550 - 49ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 64.0824 - 47ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 62.2971 - 48ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 56.4550 - 48ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 55.8447 - 51ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 55.7978 - 47ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 53.4832 - 48ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 54.9456 - 47ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 57.4451 - 47ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 51.6934 - 47ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 51.9587 - 46ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 52.1168 - 48ms/epoch - 2

23/23 - 0s - loss: 52.0410 - 47ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 49.1206 - 49ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 53.7885 - 48ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 50.3457 - 48ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 51.0216 - 48ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 56.6452 - 48ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 49.8181 - 49ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 51.4742 - 49ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 49.4099 - 48ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 48.4521 - 48ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 51.0679 - 49ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 50.3027 - 48ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 51.3506 - 50ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 50.4451 - 48ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 47.6122 - 48ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 46.7246 - 49ms/epoch - 2ms/step
Epoch 18/50

...

23/23 - 0s - loss: 35.9285 - 46ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 37.6090 - 47ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 30.0895 - 47ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 26.9455 - 47ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 26.5655 - 47ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 28.4737 - 47ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 25.9152 - 47ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 31.2641 - 48ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 28.1564 - 48ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 28.3774 - 47ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 26.7533 - 48ms/epoch - 2ms/step
Epoch 47/50
23/23 - 0s - loss: 26.3102 - 47ms/epoch - 2ms/step
Epoch 48/50
23/23 - 0s - loss: 26.3743 - 47ms/epoch - 2ms/step
Epoch 49/50
23/23 - 0s - loss: 27.4981 - 47ms/epoch - 2ms/step
Epoch 50/50
23/23 - 0s - loss: 28.9842 - 48ms/epoch - 2ms/step
10/10 - 0s - loss: 37.5523 - 47ms/epoch - 5ms/step
MSE 38: 37.55233

Epoch 11/50
23/23 - 0s - loss: 27.2753 - 50ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 28.3936 - 48ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 27.3907 - 47ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 28.4655 - 48ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 26.3615 - 47ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 28.3308 - 47ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 26.8276 - 46ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 28.6196 - 47ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 30.6912 - 48ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 29.9055 - 47ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 29.4996 - 48ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - loss: 26.3799 - 47ms/epoch - 2ms/step
Epoch 23/50
23/23 - 0s - loss: 26.0208 - 48ms/epoch - 2ms/step
Epoch 24/50
23/23 - 0s - loss: 29.3042 - 48ms/epoch - 2ms/step
Epoch 25/50
23/23 - 0s - loss: 26.9996 - 48ms/epoch - 2ms/step
Epoch 26/50
23/23 - 0s - loss: 29.4019 - 48ms/epoch - 2

Epoch 38/50
23/23 - 0s - loss: 28.5048 - 48ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 27.1763 - 48ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 26.0556 - 47ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 25.7644 - 47ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 27.1507 - 47ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 31.5596 - 52ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 27.3641 - 49ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 26.7223 - 48ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 27.1578 - 47ms/epoch - 2ms/step
Epoch 47/50
23/23 - 0s - loss: 28.0296 - 47ms/epoch - 2ms/step
Epoch 48/50
23/23 - 0s - loss: 25.5440 - 47ms/epoch - 2ms/step
Epoch 49/50
23/23 - 0s - loss: 25.7791 - 47ms/epoch - 2ms/step
Epoch 50/50
23/23 - 0s - loss: 28.4858 - 47ms/epoch - 2ms/step
10/10 - 0s - loss: 37.0068 - 47ms/epoch - 5ms/step
MSE 43: 37.00682830810547
10/10 [==============================] - 0s 3ms/step
Epoch 1/50
23/23 - 0s - loss: 27.5239 - 51ms/epoch 

Epoch 13/50
23/23 - 0s - loss: 32.3205 - 47ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 27.2834 - 47ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 25.4190 - 48ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 26.4630 - 48ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 26.2337 - 48ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 29.6115 - 49ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 30.2374 - 48ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 26.7372 - 47ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 26.5053 - 47ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - loss: 28.2235 - 48ms/epoch - 2ms/step
Epoch 23/50
23/23 - 0s - loss: 29.0502 - 48ms/epoch - 2ms/step
Epoch 24/50
23/23 - 0s - loss: 26.8153 - 48ms/epoch - 2ms/step
Epoch 25/50
23/23 - 0s - loss: 26.8184 - 48ms/epoch - 2ms/step
Epoch 26/50
23/23 - 0s - loss: 27.1108 - 48ms/epoch - 2ms/step
Epoch 27/50
23/23 - 0s - loss: 28.9935 - 48ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 24.9171 - 52ms/epoch - 2

23/23 - 0s - loss: 25.5639 - 49ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 28.7668 - 48ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 26.2827 - 47ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 28.2270 - 47ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 25.4209 - 47ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 28.7019 - 49ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 25.5845 - 48ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 28.2012 - 47ms/epoch - 2ms/step
Epoch 47/50
23/23 - 0s - loss: 25.8187 - 48ms/epoch - 2ms/step
Epoch 48/50
23/23 - 0s - loss: 26.3851 - 48ms/epoch - 2ms/step
Epoch 49/50
23/23 - 0s - loss: 26.7426 - 47ms/epoch - 2ms/step
Epoch 50/50
23/23 - 0s - loss: 27.7135 - 48ms/epoch - 2ms/step
10/10 - 0s - loss: 36.9192 - 47ms/epoch - 5ms/step
MSE 48: 36.919151306152344
10/10 [==============================] - 0s 3ms/step
Epoch 1/50
23/23 - 0s - loss: 25.6257 - 68ms/epoch - 3ms/step
Epoch 2/50
23/23 - 0s - loss: 26.9884 - 47ms/epoch - 2ms/step
E

In [20]:
# print mse and standard deviation

print('Mean: '+ str(mean))
print('Standard Deviation: '+str(standard_deviation))

Mean: 48.011555368723286
Standard Deviation: 12.292228319047842
